<a href="https://colab.research.google.com/github/rkj26/textClassifier/blob/master/webScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
!pip install selenium

In [0]:
import re
from string import ascii_uppercase
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [0]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [0]:
ports = []
vessels = []
companies = []

In [0]:
temp = 'https://directories.lloydslist.com/port-browse-name/searchid/0/searchchar/'  #This part is link of the search query that is constant so I decided to hardcode it
for c in ascii_uppercase:
  i = 1
  url = temp + c # Go over all the letters in alphabets and store the ports
  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  while(True):              # As each page has different number of results, using try and except to break the loop when there are no results left to store.
    try:
      if i == 1:
        wd.get(url=url)
        souplevel = BeautifulSoup(wd.page_source, 'lxml')
      else:
        button = wd.find_element_by_link_text(str(i))
        button.click()
        souplevel = BeautifulSoup(wd.page_source, 'lxml')

      for child in souplevel.find_all('a', href=re.compile('port-browse-listing/portid')): #All ports are stored using this template therefore used regular expression to find all <a href = ''></a> tags and stored their text. 
        if child.text != 'Learn More':
          ports.append(child.text) 
    
      i+=1
    except:
      break

In [0]:
NO_OF_PAGES=25 #Only 25 pages to view therefore hard coded it.
wd = webdriver.Chrome('chromedriver',options=chrome_options)
url = 'https://www.marinetraffic.com/en/data/?asset_type=vessels&columns=flag,shipname,photo,recognized_next_port,reported_eta,reported_destination,current_port,imo,ship_type,show_on_live_map,time_of_latest_position,lat_of_latest_position,lon_of_latest_position'
wd.get(url=url)
for i in range(NO_OF_PAGES):
  button = WebDriverWait(wd, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="reporting_ag_grid"]/div/div[2]/div[3]/div/div/div/div/div[3]/button[2]')))
  if i!=0:
    button.click()
  soup = BeautifulSoup(wd.page_source, 'lxml')
  for element in soup.find_all('a', href=re.compile('/en/ais/details/ships/')): #Similar thought as the for port web scrapper.
    vessels.append(element.text)

In [0]:
url = 'https://opencorporates.com/registers'
wd = webdriver.Chrome('chromedriver',options=chrome_options)
wd.get(url)
soup = BeautifulSoup(wd.page_source, 'lxml')
numberOfRegistries = int(soup.find(id='DataTables_Table_0_info').text.split(' ')[-2]) #Finding out the number of registries
for i in range(numberOfRegistries):
  xpath = '//*[@id="DataTables_Table_0"]/tbody/tr['+str(i+1)+']/td[3]/a' #Every row of the table and third column which is a clickable link to the companies.
  companiesLink = WebDriverWait(wd, 20).until(EC.presence_of_element_located((By.XPATH, xpath)))
  companiesLink.click()
  lookup = str(wd.current_url)[26:]+'/'  #Lookup for the directory link which is used for every company in the registry

  for j in range(3): 
    if j != 0:
      nextPath = '//*[@id="results"]/div/div[1]/ul/li[8]/a'  #XPath for the next button
      nextButton = wd.find_element_by_xpath(nextPath)
      nextButton.click()
        
    subSoup = BeautifulSoup(wd.page_source, 'lxml')
    for k in subSoup.find_all('a', href=re.compile(lookup)):
      companies.append(k.text)
  wd.get(url)

In [0]:
ports = np.array(ports)
vessels = np.array(vessels)
companies = np.array(companies)

In [9]:
print(len(ports))
print(len(vessels))
print(len(companies))

6657
500
12512
